In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

def plot_central_row(image, title, color):
    # Obtener la fila central de la imagen
    row = image.shape[0] // 2
    values = image[row, :]

    # Dibujar la gráfica de la fila central
    line, = plt.plot(values, label=title, color=color)
    return line

def main():
    # Cargar la imagen en escala de grises
    image = cv2.imread('/home/alejandro/Pictures/duna.jpg', cv2.IMREAD_GRAYSCALE)
    image = cv2.resize(image, (512, 512))
    
    # Convertir la imagen a un rango [0, 1] para procesamiento
    image = image.astype(np.float32) / 255.0

    # Agregar ruido a la imagen
    sigma_noise = 1e-1
    noise = np.random.normal(0, sigma_noise, image.shape)
    noisy_image = image + noise

    # Hard-thresholding
    dct = cv2.dct(np.float32(noisy_image))  # Aplicar DCT
    hard_threshold = 3.2 * sigma_noise
    dct[np.abs(dct) <= hard_threshold] = 0  # Umbralización dura
    hard_cleanf = cv2.idct(dct)  # Inversa de DCT

    # Soft-thresholding
    soft_dct = dct.copy()
    soft_threshold = 1.5 * sigma_noise
    soft_dct[np.abs(soft_dct) <= 1e-6] = 0  # Eliminar valores muy pequeños
    soft_dct = np.sign(soft_dct) * np.maximum(0, np.abs(soft_dct) - soft_threshold)  # Reducir coeficientes pequeños
    soft_cleanf = cv2.idct(soft_dct)  # Inversa de DCT

    # Convertir imágenes a [0, 255] para visualización
    noisy_image = np.uint8(np.clip(noisy_image * 255, 0, 255))
    image = np.uint8(np.clip(image * 255, 0, 255))
    hard_cleanf = np.uint8(np.clip(hard_cleanf * 255, 0, 255))
    soft_cleanf = np.uint8(np.clip(soft_cleanf * 255, 0, 255))

    # Mostrar imágenes con OpenCV
    cv2.imshow('Original Image', image)
    cv2.imshow('Noisy Image', noisy_image)
    cv2.imshow('Hard-Threshold Denoised', hard_cleanf)
    cv2.imshow('Soft-Threshold Denoised', soft_cleanf)

    # Mostrar el PSNR de cada imagen
    print(f"Noisy image PSNR: {cv2.PSNR(noisy_image, image):.2f}")
    print(f"Reconstruction (Hard) PSNR: {cv2.PSNR(hard_cleanf, image):.2f}")
    print(f"Reconstruction (Soft) PSNR: {cv2.PSNR(soft_cleanf, image):.2f}")

    # Graficar la fila central de cada imagen
    plt.figure(figsize=(18, 10), dpi=700)  # Aumentar el tamaño de la figura para mayor resolución
    line1 = plot_central_row(image, "Original", 'blue')
    line2 = plot_central_row(noisy_image, "Noisy", 'red')
    line3 = plot_central_row(hard_cleanf, "Hard Denoised", 'green')
    line4 = plot_central_row(soft_cleanf, "Soft Denoised", 'purple')

    # Mejorar el zoom de la gráfica para que se vea más detallada
    plt.xlim(0, image.shape[1])  # Asegurarse de que el eje x cubra todo el ancho de la imagen
    plt.ylim(0, 255)  # Limitar el eje y para que cubra el rango de intensidades [0, 255]

    # Personalizar la gráfica
    plt.legend()
    plt.title("Central Row Intensity Profile")
    plt.xlabel("Pixel Position")
    plt.ylabel("Intensity")

    # Mostrar la gráfica
    plt.show()

    # Esperar una tecla para cerrar las ventanas de OpenCV
    cv2.waitKey(0)
    cv2.destroyAllWindows()

if __name__ == '__main__':
    main()


Noisy image PSNR: 20.19
Reconstruction (Hard) PSNR: 28.59
Reconstruction (Soft) PSNR: 28.72


: 